# ND bunch/point cloud analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ultraplot as uplt
from ipywidgets import interact
from ipywidgets import widgets

import psdist as ps
import psdist.plot as psv

In [ ]:
uplt.rc["cmap.discrete"] = False
uplt.rc["colorbar.width"] = "1.2em"
uplt.rc["cycle"] = "538"
uplt.rc["cmap.sequential"] = "viridis"
uplt.rc["figure.facecolor"] = "white"
uplt.rc["grid"] = False

## Make distribution

In [ ]:
def gen_points(ndim: int, size: int, seed: int = 0) -> np.ndarray:    
    rng = np.random.default_rng(seed)
    
    cov_matrix = np.identity(ndim)
    for i in range(ndim):
        for j in range(i):
            cov_matrix[i, j] = cov_matrix[j, i] = rng.uniform(-0.2, 0.2)
    
    points = rng.multivariate_normal(np.zeros(ndim), cov_matrix, size=size)
    for _ in range(4):
        loc = rng.uniform(-3.0, +3.0, size=ndim)
        scale = rng.uniform(+0.5, +1.5, size=ndim)
        points = np.vstack([points, rng.normal(loc=loc, scale=scale, size=(size, ndim))])

    points = points - np.mean(points, axis=0)
    return points

## 2D plotting

In [ ]:
points = gen_points(ndim=6, size=1000, seed=0)
points = points[:, (0, 1)]
dims = ["x", "px"]

In [ ]:
cmap = uplt.Colormap("mono", left=0.1)
kinds = ["scatter", "hist", "contour", "contourf", "kde"]

fig, axs = uplt.subplots(ncols=len(kinds), figwidth=8.0)
for ax, kind in zip(axs, kinds):
    kws = dict()
    if kind == "scatter":
        kws["c"] = cmap(1.0)
        kws["s"] = 1.0
    else:
        kws["cmap"] = cmap

    if kind == "kde":
        kws["bins"] = 100
        
    psv.points.plot(
        points,
        ax=ax,
        kind=kind,
        rms_ellipse=True,
        rms_ellipse_kws=dict(level=2.0, color="red"),
        **kws,
    )
    ax.format(title=kind)
axs.format(xlabel=dims[0], ylabel=dims[1])
plt.show()

### Enclosing sphere/ellipsoid 

In [ ]:
fig, ax = uplt.subplots(figwidth=None)
psv.points.plot(points, ax=ax, kind="scatter", c="gray", s=3.0)

fractions = np.linspace(0.10, 1.0, 3)
cmap = uplt.Colormap("reds", left=0.2, reverse=True)
for fraction in fractions:
    level = ps.points.enclosing_ellipsoid_radius(points, fraction=fraction)
    psv.points.plot_rms_ellipse(
        points,
        ax=ax,
        level=level,
        color=cmap(fraction),
        lw=1.5,
        label=f"{(100.0 * fraction):.0f}%",
    )
ax.legend(loc="t", ncols=3, framealpha=0)
ax.format(xlabel=dims[0], ylabel=dims[1])
plt.show()

### Spherical and ellipsoidal shell slices 

In [ ]:
rmin = 2.0
rmax = 3.0
points_slice = ps.points.slice_sphere(points, rmin=rmin, rmax=rmax)

fig, ax = uplt.subplots(figwidth=None)
psv.points.plot(points,       ax=ax, kind="scatter", color="grey")
psv.points.plot(points_slice, ax=ax, kind="scatter", color="red")
for r in [rmin, rmax]:
    psv.plot_circle(r=r, ax=ax)
ax.format(xlabel=dims[0], ylabel=dims[1])
plt.show()

In [ ]:
rmin = 1.0
rmax = 1.5
points_slice = ps.points.slice_ellipsoid(points, rmin=rmin, rmax=rmax)

fig, ax = uplt.subplots(figwidth=None)
psv.points.plot(points,       ax=ax, kind="scatter", color="grey")
psv.points.plot(points_slice, ax=ax, kind="scatter", color="red")
for r in [rmin, rmax]:
    psv.points.plot_rms_ellipse(points, level=r, ax=ax)
ax.format(xlabel=dims[0], ylabel=dims[1])
plt.show()

In [ ]:
@interact(radius=widgets.FloatRangeSlider(min=0.0, max=5.0))
def update(radius):
    (rmin, rmax) = radius
    points_slice = ps.points.slice_ellipsoid(
        points, axis=(0, 1), rmin=rmin, rmax=rmax
    )

    fig, ax = uplt.subplots(figwidth=None)
    psv.points.plot(points,       ax=ax, kind="scatter", color="grey")
    psv.points.plot(points_slice, ax=ax, kind="scatter", color="red")
    for r in [rmin, rmax]:
        psv.points.plot_rms_ellipse(points, level=r, ax=ax)
    ax.format(xlabel=dims[0], ylabel=dims[1])
    plt.show()

### Contour shell slice

In [ ]:
points = gen_points(ndim=6, size=1_000_000, seed=0)
points = points[:, (0, 1)]
    
bins = 45

fig, ax = uplt.subplots()
psv.plot_points(
    points,
    ax=ax,
    kind="hist",
    process_kws=dict(scale_max=True),
    bins=bins,
    cmap=uplt.Colormap("mono", left=0.0, right=1.0),
    colorbar=True,
    discrete=False,
)
for lmin, color in zip([0.0, 0.2, 0.8], ["red3", "red7", "darkred"]):
    lmax = lmin + 0.15
    psv.plot_points(
        ps.points.downsample(
            ps.points.slice_contour(
                points, lmin=lmin, lmax=lmax, hist_kws=dict(bins=bins)
            ),
            size=1000,
        ),
        ax=ax,
        kind="scatter",
        color=color,
        s=1.0,
    )
ax.format(xlabel=dims[0], ylabel=dims[1])
plt.show()

### Joint plot 

In [ ]:
grid = psv.JointGrid(
    marg_kws=dict(space="2.0em", width="7.0em"),
    marg_fmt_kws_x=dict(yspineloc="left", xspineloc="bottom"),
    marg_fmt_kws_y=dict(yspineloc="left", xspineloc="bottom"),
    xspineloc="bottom",
    yspineloc="left",
)
grid.plot_points(
    points[:, axis],
    marg_kws=dict(kind="step", color="black", scale="max"),
    marg_hist_kws=dict(bins=75),
    kind="hist",
    process_kws=dict(norm="max"),
    cmap=uplt.Colormap("blues", left=0.0),
    discrete=False,
    norm="log",
    colorbar=True,
    colorbar_kw=dict(width="1.2em", tickminor=True),
)
psv.points.plot(
    points[:, axis],
    ax=grid.ax,
    process_kws=dict(norm="max", blur=5.0),
    kind="contour",
    levels=[0.001, 0.01, 0.1, 1.0],
    colors="black",
    lw=0.4,
)
ymin = 10.0**-5.0
ymax = None
grid.ax_marg_x.format(yformatter="log", yscale="log", ymin=ymin, ymax=ymax)
grid.ax_marg_y.format(xformatter="log", xscale="log", xmin=ymin, xmax=ymax)
grid.ax.format(xlabel=_dims[0], ylabel=_dims[1])
plt.show()

## Corner plot 

In [ ]:
ndim = 6
size = 1_000_000
dims = ["x", "px", "y", "py", "z", "pz"]

In [ ]:
alpha = 0.5
cmaps = [uplt.Colormap(name, left=0.2) for name in ["blues", "reds", "greens"]]
colors = [cmap(0.75) for cmap in cmaps]
plot_kws = dict(
    mask=True,
    alpha=alpha,
    rms_ellipse=True,
)
autolim_kws = dict(pad=0.1, zero_center=True)

In [ ]:
grid = psv.CornerGrid(points.shape[1], diag=True, corner=False, figwidth=7.0)
grid.set_labels(dims)

X = points.copy()
Y = X[:100_000].copy()
R = ps.ap.phase_adv_matrix(np.radians(45.0), np.radians(20.0), np.radians(-45.0))
for i in range(3):
    if i > 0:
        Y = ps.points.transform_linear(Y, R)
        Y = Y + rng.uniform(-3.5, 3.5, size=(1, Y.shape[1]))
        Y[:, 3] += 0.01 * Y[:, 1] ** 3 - 0.0002 * Y[:, 2] ** 4
    grid.plot_points(
        Y,
        kind="hist",
        bins="auto",
        lower=True,
        upper=False,
        diag_kws=dict(color=colors[i], alpha=alpha),
        cmap=cmaps[i],
        **plot_kws,
    )
    grid.plot_points(
        Y,
        kind="contour",
        bins=25,
        lower=False,
        upper=True,
        diag_kws=dict(alpha=0.0),
        cmap=cmaps[i],
        **plot_kws,
    )
plt.show()

### Normalized radial histogram

Normalize by the volume of each spherical shell.

In [ ]:
profile, edges = ps.points.radial_histogram(points[:, :4], bins=50)
profile = psv.scale_profile(profile, scale="max")
coords = ps.utils.coords_from_edges(edges)

fig, ax = uplt.subplots()
psv.plot_profile(profile, edges=edges, ax=ax, kind="step", color="black", label="data")
ax.plot(coords, np.exp(-0.5 * coords**2), color="red5", alpha=0.5, label="gaussian")
ax.legend()
ax.format(
    yscale="log",
    yformatter="log",
    ymin=0.0001,
    xlabel="Radius",
    ylabel="Normalized density",
)
plt.show()

Define $\mathbf{r} = [x, y, z]^T$, $\mathbf{r}' = [p_x, p_y, p_z]^T$.

In [ ]:
r = ps.points.get_radii(X[:, (0, 2, 4)])
pr = ps.points.get_radii(X[:, (1, 3, 5)])
R = np.vstack([r, pr]).T

grid = psv.JointGrid(
    marg_kws=dict(space="2.0em", width="7.0em"),
    marg_fmt_kws_x=dict(yspineloc="left", xspineloc="bottom", ylabel="f(r)"),
    marg_fmt_kws_y=dict(yspineloc="left", xspineloc="bottom", xlabel="f(pr)"),
)
grid.plot_points(
    R,
    marg_kws=dict(scale="max", fill=True, color="black", alpha=0.3),
    bins=100,
    process_kws=dict(norm="max"),
    offset=1.0,
    N=21,
    norm="log",
    vmax=1.0,
    cmap="blues",
    colorbar=True,
    colorbar_kw=dict(label="f(r, pr)"),
)
grid.ax.format(xlabel="r", ylabel="pr")
plt.show()

### Sparse histogram

In [ ]:
edges = ps.points.histogram_bin_edges(X, bins=6)
nonzero_indices, nonzero_counts, nonzero_edges = ps.points.sparse_histogram(
    points, bins=edges
)
print(f"Nonzero bins from sparse hist: {len(nonzero_counts)}")

In [ ]:
hist, _ = ps.points.histogram(points, bins=edges)
print(f"Nonzero bins from hist: {hist[hist > 0].size}")

## Slice matrix plot 

[To do...]

## Interactive slicing

### 1D 

In [ ]:
X = points.copy()
X = X - np.mean(X, axis=0)
Y = Y - np.mean(Y, axis=0)
data = [
    [Y, X, 2.0 * X],
    [X, -Y, -2.0 * Y],
]

In [ ]:
psv.points.interactive_slice_1d(
    data=data,
    dims=dims,
    slice_type="range",  # {"int", "range"}
    options=dict(
        alpha=True,
        auto_plot_res=False,
        kind=True,
        log=True,
        normalize=False,
        scale=False,
    ),
    share_limits=False,
    legend=True,
    labels=None,
    cycle="538",
    fig_kws=dict(figsize=(5.0, 2.0)),
)

### 2D 

In [ ]:
plot_kws = dict(
    profx=True,
    profy=True,
    prof_kws=dict(color="white"),
    process_kws=dict(norm="max"),
    autolim_kws=dict(pad=0.1),
    offset=1.0,
    colorbar=True,
    colorbar_kw=dict(tickminor=True, width="1.2em"),
)

In [ ]:
psv.points.interactive_slice_2d(
    data=data,
    limits=None,
    share_limits=1,
    default_axis=(0, 1),
    slice_type="range",
    plot_res=64,
    slice_res=16,
    dims=dims,
    units=None,
    options=dict(
        auto_plot_res=False,
        discrete=True,
        ellipse=True,
        log=True,
        mask=False,
        normalize=True,
        profiles=True,
    ),
    fig_kws=dict(toplabels=["left", "right"], space=6.0),
    **plot_kws,
)